In [88]:
import pandas as pd

pl = r"C:\Users\mkreidler\Desktop\35 wacker ctrac to zoda\35w Packing List 2019-04-16.xlsx"
ct = r"C:\Users\mkreidler\Desktop\35 wacker ctrac to zoda\35 wacker with block num.xlsx"


packing = pd.read_excel(pl)
ctrac = pd.read_excel(ct)
packing = packing[packing["CROSSED OFF"].isna()]
packing["cnt"] = packing.groupby("Block Style").cumcount() + 1
packing["cc"] = packing["Block Style"] + "#" + packing.cnt.map(str)

packing.head()

,Entry,UNIT NUM,Block Style,Crate,CROSSED OFF,note,cnt,cc
1,2,2522.0,9F3,48.0,NaN,NaN,1,9F3#1
2,3,2521.0,9F3,48.0,NaN,NaN,2,9F3#2
3,4,2438.0,9E5,48.0,NaN,NaN,1,9E5#1
4,5,2440.0,9E5,48.0,NaN,NaN,2,9E5#2
5,6,2534.0,9F5,48.0,NaN,NaN,1,9F5#1


In [89]:
import re

ctrac["pre"] = ""
ctrac["prior_digit"] = ""
ctrac["elev_letter"] = ""


def get_digit(x):
    x = str(x)
    if x == "SHIPPED":
        return "Shipped"
    else:
        pat = r"(\d{1,10})"
        result = re.search(pattern=pat, string=x)
        if result:
            return result.group(1)
        
        
def get_letter(x):
    pat = r"[\d]{1,6}([a-zA-Z]{1,20})"
    x = str(x)
    result = re.search(pat, x)
    if result:
        return result.group(1)
    

ctrac.prior_digit = ctrac.Priority.apply(get_digit)
ctrac.elev_letter = ctrac.Priority.apply(get_letter)

ctrac.pre = ctrac.prior_digit + ctrac.elev_letter


c = ctrac[["Priority", "elev_letter", "prior_digit"]]

c[c.Priority == "SHIPPED"].head()


,Priority,elev_letter,prior_digit
430,SHIPPED,None,Shipped
431,SHIPPED,None,Shipped
432,SHIPPED,None,Shipped
433,SHIPPED,None,Shipped
434,SHIPPED,None,Shipped


In [90]:
###FILTER OUT CTRAC FOR WHAT WE ARE LOOKING FOR

ctrac = ctrac[ctrac["Void Status"].isna() == True]
ctrac.Priority.unique()

array(['7W_3', '6W_3', '3S_3', '2S_3', '2E_3', '1S_3', '1E_3', '6S_D5_3',
       '5S_3', '4S_3', 'ZEXTRA', '6S_D5_2', '6S_D5_1', '3E_1', '9W_2',
       '6W_D2_2', '4W_2', '9W_1', '8S_1', '6W_D2_1', '6W_D1_1', '5S_1',
       '4W_1', '4S_1', '3S_1', '3E_2', '1E_1', '2E_2', '2E_1', '1S_1',
       '1E_2', '11W_1', '1S_1_RE', 'SHIPPED', '9W', '9W_RE', '9S',
       '9S_RE', '9S_1_RE', '8W_RE', '8W', '8S_RE', '8S', '8S_1_RE',
       '7W_RE', '7W', '7W_1_RE', '7S', '7S_1_RE', '6W_D1_RE', '6W_D2',
       '6W_D1', 'NIS', '6S_D5_RE', '6S_D5', '6S_D4', '6S_1_RE', '5W_RE',
       '5W', '5S_RE', '5S', '5S_1_RE', '4W_RE', '4W', '4S_RE', '4S',
       '4S_1_RE', '3S_RE', '3S', '3E_RE', '3E', '2S_RE', '2S', '2S_1_RE',
       '2E_RE', '2E', '1S_RE', '1S', '1E_RE', '1E', '13S', '13S_RE',
       '13S_1_RE', '12S_D2', '12S_D3RE', '12S_D2RE', '12S_D3', '12S_1_RE',
       '11W_RE', '11W', '11W_1_RE', '11S', '11S_1_RE', '10W', '10W_RE',
       '10W_1_RE', '10S', '10S_RE', '10S_1_RE'], dtype=object)

In [91]:
list(ctrac)

['Priority',
 'Block ID',
 'FormingMethod',
 'SampleName',
 'DrawingNumber',
 'UnitQuantity',
 'Crate Number',
 'Current Location',
 'Current Shipping Sta',
 'Void Status',
 'Date Scanned',
 'Block Line Number',
 'pre',
 'prior_digit',
 'elev_letter']

In [92]:
ctrac.pre.unique()

array(['7W', '6W', '3S', '2S', '2E', '1S', '1E', '6S', '5S', '4S', nan,
       '3E', '9W', '4W', '8S', '11W', '9S', '8W', '7S', '5W', '13S',
       '12S', '11S', '10W', '10S'], dtype=object)

In [93]:
ctrac[ctrac.pre.isna()].head()

,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta,Void Status,Date Scanned,Block Line Number,pre,prior_digit,elev_letter
64,ZEXTRA,8C8,RAM,NaN,35WS-8C8,1,0,FINIS,NaN,NaN,20190401,3068,NaN,None,None
65,ZEXTRA,8C8,RAM,NaN,35WS-8C8,1,0,FINIS,NaN,NaN,20190401,3067,NaN,None,None
66,ZEXTRA,8C2,RAM,NaN,35WS-8C2,1,0,FINIS,NaN,NaN,20190402,3066,NaN,None,None
67,ZEXTRA,8C2,RAM,NaN,35WS-8C2,1,0,FINIS,NaN,NaN,20190402,3065,NaN,None,None
68,ZEXTRA,8C1,RAM,NaN,35WS-8C1,1,0,FINIS,NaN,NaN,20190401,3064,NaN,None,None


In [94]:
ctrac["surveyed"] = ""

def surveyed(x):
    exclude = ["7S", "8S", "9S", "10S", "11S", "12S", "13S"]
    if x not in exclude:
        return "Surveyed"
    
ctrac.surveyed = ctrac.pre.apply(surveyed)

ctrac.head()

,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta,Void Status,Date Scanned,Block Line Number,pre,prior_digit,elev_letter,surveyed
0,7W_3,8C6A,RAM,8/36,35E-68-5,1,0,NaN,NaN,NaN,0,3132,7W,7,W,Surveyed
1,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3131,7W,7,W,Surveyed
2,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3130,7W,7,W,Surveyed
3,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3129,7W,7,W,Surveyed
4,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3128,7W,7,W,Surveyed


In [95]:
ctrac.Priority.unique()

array(['7W_3', '6W_3', '3S_3', '2S_3', '2E_3', '1S_3', '1E_3', '6S_D5_3',
       '5S_3', '4S_3', 'ZEXTRA', '6S_D5_2', '6S_D5_1', '3E_1', '9W_2',
       '6W_D2_2', '4W_2', '9W_1', '8S_1', '6W_D2_1', '6W_D1_1', '5S_1',
       '4W_1', '4S_1', '3S_1', '3E_2', '1E_1', '2E_2', '2E_1', '1S_1',
       '1E_2', '11W_1', '1S_1_RE', 'SHIPPED', '9W', '9W_RE', '9S',
       '9S_RE', '9S_1_RE', '8W_RE', '8W', '8S_RE', '8S', '8S_1_RE',
       '7W_RE', '7W', '7W_1_RE', '7S', '7S_1_RE', '6W_D1_RE', '6W_D2',
       '6W_D1', 'NIS', '6S_D5_RE', '6S_D5', '6S_D4', '6S_1_RE', '5W_RE',
       '5W', '5S_RE', '5S', '5S_1_RE', '4W_RE', '4W', '4S_RE', '4S',
       '4S_1_RE', '3S_RE', '3S', '3E_RE', '3E', '2S_RE', '2S', '2S_1_RE',
       '2E_RE', '2E', '1S_RE', '1S', '1E_RE', '1E', '13S', '13S_RE',
       '13S_1_RE', '12S_D2', '12S_D3RE', '12S_D2RE', '12S_D3', '12S_1_RE',
       '11W_RE', '11W', '11W_1_RE', '11S', '11S_1_RE', '10W', '10W_RE',
       '10W_1_RE', '10S', '10S_RE', '10S_1_RE'], dtype=object)

In [96]:
ctrac["scope"] = ""
def remove_priorities(x):
    exclude = ["ZEXTRA", "SHIPPED", "NIS", "6S_D4"]

    if x not in exclude:
        return "in scope"
        
        
ctrac.scope = ctrac.Priority.apply(remove_priorities)

ctrac.head()

,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta,Void Status,Date Scanned,Block Line Number,pre,prior_digit,elev_letter,surveyed,scope
0,7W_3,8C6A,RAM,8/36,35E-68-5,1,0,NaN,NaN,NaN,0,3132,7W,7,W,Surveyed,in scope
1,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3131,7W,7,W,Surveyed,in scope
2,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3130,7W,7,W,Surveyed,in scope
3,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3129,7W,7,W,Surveyed,in scope
4,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3128,7W,7,W,Surveyed,in scope


In [97]:
ctrac = ctrac[ctrac.scope.isna() == False]
ctrac = ctrac[ctrac.surveyed.isna() == False]

ctrac = ctrac[ctrac["Date Scanned"] == 0]

ctrac


,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta,Void Status,Date Scanned,Block Line Number,pre,prior_digit,elev_letter,surveyed,scope
0,7W_3,8C6A,RAM,8/36,35E-68-5,1,0,NaN,NaN,NaN,0,3132,7W,7,W,Surveyed,in scope
1,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3131,7W,7,W,Surveyed,in scope
2,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3130,7W,7,W,Surveyed,in scope
3,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3129,7W,7,W,Surveyed,in scope
4,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3128,7W,7,W,Surveyed,in scope
5,7W_3,8C5A,RAM,NaN,35E-67-1,1,0,NaN,NaN,NaN,0,3127,7W,7,W,Surveyed,in scope
6,7W_3,8C4A,RAM,NaN,35E-66-1,1,0,NaN,NaN,NaN,0,3126,7W,7,W,Surveyed,in scope
7,7W_3,8C4,RAM,6,35E-66,1,0,NaN,NaN,NaN,0,3125,7W,7,W,Surveyed,in scope
8,7W_3,8C4,RAM,6,35E-66,1,0,NaN,NaN,NaN,0,3124,7W,7,W,Surveyed,in scope
9,7W_3,8C4,RAM,6,35E-66,1,0,NaN,NaN,NaN,0,3123,7W,7,W,Surveyed,in scope


In [98]:
#ctrac.to_excel(r"C:\Users\mkreidler\Desktop\35 wacker ctrac to zoda\result.xlsx")

In [99]:
ctrac["cnt"] = ctrac.groupby("Block ID").cumcount() +1
ctrac["cc"] = ctrac["Block ID"] + "#" +ctrac.cnt.map(str)

ctrac.head()

,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta,Void Status,Date Scanned,Block Line Number,pre,prior_digit,elev_letter,surveyed,scope,cnt,cc
0,7W_3,8C6A,RAM,8/36,35E-68-5,1,0,NaN,NaN,NaN,0,3132,7W,7,W,Surveyed,in scope,1,8C6A#1
1,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3131,7W,7,W,Surveyed,in scope,1,8C6#1
2,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3130,7W,7,W,Surveyed,in scope,2,8C6#2
3,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3129,7W,7,W,Surveyed,in scope,3,8C6#3
4,7W_3,8C6,HP,8/36,35E-68,1,0,NaN,NaN,NaN,0,3128,7W,7,W,Surveyed,in scope,4,8C6#4


In [105]:
merge = ctrac.merge(packing, left_on="Block Line Number", right_on="UNIT NUM", how="outer")

len(merge)

524

In [101]:
merge.head()

,Priority,Block ID,FormingMethod,SampleName,DrawingNumber,UnitQuantity,Crate Number,Current Location,Current Shipping Sta,Void Status,...,cnt_x,cc_x,Entry,UNIT NUM,Block Style,Crate,CROSSED OFF,note,cnt_y,cc_y
0,7W_3,8C6A,RAM,8/36,35E-68-5,1.0,0.0,NaN,NaN,NaN,...,1.0,8C6A#1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7W_3,8C6,HP,8/36,35E-68,1.0,0.0,NaN,NaN,NaN,...,1.0,8C6#1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7W_3,8C6,HP,8/36,35E-68,1.0,0.0,NaN,NaN,NaN,...,2.0,8C6#2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7W_3,8C6,HP,8/36,35E-68,1.0,0.0,NaN,NaN,NaN,...,3.0,8C6#3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7W_3,8C6,HP,8/36,35E-68,1.0,0.0,NaN,NaN,NaN,...,4.0,8C6#4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
list(merge)

['Priority',
 'Block ID',
 'FormingMethod',
 'SampleName',
 'DrawingNumber',
 'UnitQuantity',
 'Crate Number',
 'Current Location',
 'Current Shipping Sta',
 'Void Status',
 'Date Scanned',
 'Block Line Number',
 'pre',
 'prior_digit',
 'elev_letter',
 'surveyed',
 'scope',
 'cnt_x',
 'cc_x',
 'Entry',
 'UNIT NUM',
 'Block Style',
 'Crate',
 'CROSSED OFF',
 'note',
 'cnt_y',
 'cc_y']

In [103]:
merge.to_excel(r"C:\Users\mkreidler\Desktop\35 wacker ctrac to zoda\merge.xlsx")